In [34]:
import pandas as pd
# Carregando dados
import glob

def df_all_folder(path,sufixo="_tweets.csv"):
    all_files = glob.glob(path + "*"+sufixo)
    li = []    
    
    for filename in all_files:
        df = pd.read_csv(filename, index_col=0)
        if "janones" not in filename: 
            li.append(df)
    names = []
    for filename in all_files:
        x = filename.replace(path, "")
        x = x.replace(sufixo, "")
        names.append(x.replace(" ","_"))
    return pd.concat(li, axis=0, keys =names).reset_index().rename(columns={'level_0': "candidato"})

In [35]:
df = df_all_folder("dados/ultimos_100_tweets/")
df

,candidato,level_1,text,id,retweets,replys,likes,author_id,source,lang,created_at,place_id
0,dados/ultimos_100_tweets\bivar,0,@fartosuely1 @_afadamadrinha @thinkmoreBrazil ...,1.537649e+18,1.0,0.0,1.0,1.410976e+18,Twitter for Android,pt,2022-06-17 04:11:17+00:00,NaN
1,dados/ultimos_100_tweets\bivar,1,@ElenTorres10 Quem confia no Bivar?,1.537646e+18,0.0,0.0,0.0,2.387485e+08,Twitter for iPhone,pt,2022-06-17 04:00:30+00:00,NaN
2,dados/ultimos_100_tweets\bivar,2,"Moro pode escolher cargo para concorrer, diz o...",1.537646e+18,0.0,0.0,0.0,1.487306e+18,Twitter Web App,pt,2022-06-17 03:58:56+00:00,NaN
3,dados/ultimos_100_tweets\bivar,3,"@luzia74992679 Acorda Bivar, é Moro para Presi...",1.537646e+18,0.0,0.0,0.0,1.410976e+18,Twitter for Android,pt,2022-06-17 03:58:30+00:00,NaN
4,dados/ultimos_100_tweets\bivar,4,"Bivar pensa que todos são alienados, Moro para...",1.537645e+18,1.0,1.0,2.0,1.487306e+18,Twitter Web App,pt,2022-06-17 03:54:36+00:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1831,dados/ultimos_100_tweets\sofia_manzano,95,@Anitta ANITTA VC PRECISA VER SOBRE A VERA LÚC...,1.546963e+18,0.0,0.0,0.0,1.460646e+18,Twitter for Android,pt,2022-07-12 21:02:57+00:00,NaN
1832,dados/ultimos_100_tweets\sofia_manzano,96,Olá 👋 passando aqui para informar que a partir...,1.546962e+18,0.0,0.0,4.0,4.195617e+08,Twitter Web App,pt,2022-07-12 20:57:43+00:00,NaN
1833,dados/ultimos_100_tweets\sofia_manzano,97,O dinheiro que a mona investiu pra superar a V...,1.546958e+18,0.0,1.0,1.0,1.158390e+18,Twitter for iPhone,pt,2022-07-12 20:40:36+00:00,NaN
1834,dados/ultimos_100_tweets\sofia_manzano,98,"Daqui a pouco, as 18h, a rádio web 2 de Julho ...",1.546957e+18,0.0,0.0,0.0,1.506264e+18,Twitter Web App,pt,2022-07-12 20:35:52+00:00,NaN


In [36]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\eduar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [37]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eduar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [38]:
df = df[df['text'].notna()]

In [39]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
 
def remove_stop_words(text):
    text = word_tokenize(text.lower())
    stop_words = set(stopwords.words('portuguese'))
    text = [word for word in text if word.isalpha() and not word in stop_words]
    return ' '.join(text)
     
x = df.apply(lambda row: remove_stop_words(row['text']), axis=1)
x


0       thinkmorebrazil deltanmd acorda bivar moro pre...
1                                            confia bivar
2       moro pode escolher cargo concorrer diz preside...
3                            acorda bivar moro presidente
4       bivar pensa todos alienados moro governador pr...
                              ...                        
1831    anitta anitta vc precisa ver sobre vera lúcia ...
1832    olá passando aqui informar partir semana pré p...
1833    dinheiro mona investiu pra superar vera lúcia ...
1834    daqui pouco rádio web julho reapresenta entrev...
1835    simonetebetbr tebet polarização pt x psdb bols...
Length: 1738, dtype: object

In [40]:
# x_test = x[0:int(len(x)*0.3)]
# x_trein = x[int(len(x)*0.3):len(x)]

In [41]:
y = df_all_folder("dados/ultimos_100_tweets/",sufixo='_labeled.csv')['level_1'].dropna().apply(lambda x : x.upper())
y

0       NT
1       NG
2       NG
3       NG
4       NG
        ..
1734    PS
1735    NT
1736    NT
1737    NT
1738    NT
Name: level_1, Length: 1738, dtype: object

In [42]:
y = y.apply(lambda x: 0 if x == 'NG' else 1 if x == 'NT' else 2)
y

0       1
1       0
2       0
3       0
4       0
       ..
1734    2
1735    1
1736    1
1737    1
1738    1
Name: level_1, Length: 1738, dtype: int64

In [43]:
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

def tokenizar(x):
    max_words = 20000
    
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(x)
    sequences = tokenizer.texts_to_sequences(x)
    return pad_sequences(sequences, maxlen=500)
x = tokenizar(x)

In [44]:
import numpy as np
new_y = y.copy()
new_y = new_y.apply(lambda x: 1 if x == 2 else 0)
new_y

0       0
1       0
2       0
3       0
4       0
       ..
1734    1
1735    0
1736    0
1737    0
1738    0
Name: level_1, Length: 1738, dtype: int64

In [45]:
y_test = y[0:int(len(y)*0.3)]
y_trein = y[int(len(y)*0.3):len(y)]
x_test = x[0:int(len(x)*0.3)]
x_trein = x[int(len(x)*0.3):len(x)]
y_test_n = new_y[0:int(len(new_y)*0.3)]
y_trein_n = new_y[int(len(new_y)*0.3):len(new_y)]

In [46]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Masking, Conv1D, MaxPool1D, GRU
from keras.layers import Embedding
from keras.metrics import Precision, Recall
 
model = Sequential() 
model.add(Embedding(20000, 32, input_length=500)) 
model.add(Masking(mask_value=0.0))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(32, return_sequences=False))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])

In [47]:
[print(i.shape, i.dtype) for i in model.inputs]
[print(o.shape, o.dtype) for o in model.outputs]

(None, 500) <dtype: 'float32'>
(None, 1) <dtype: 'float32'>


[None]

In [48]:
model.fit(x_trein, y_trein_n, epochs=12, verbose=1)

Epoch 1/20
39/39 [==============================] - 47s 146ms/step - loss: 0.6076 - accuracy: 0.7354
Epoch 2/20
39/39 [==============================] - 6s 153ms/step - loss: 0.5794 - accuracy: 0.7354
Epoch 3/20
39/39 [==============================] - 6s 153ms/step - loss: 0.5348 - accuracy: 0.7609
Epoch 4/20
39/39 [==============================] - 6s 154ms/step - loss: 0.3733 - accuracy: 0.8578
Epoch 5/20
39/39 [==============================] - 6s 158ms/step - loss: 0.2284 - accuracy: 0.9236
Epoch 6/20
39/39 [==============================] - 7s 168ms/step - loss: 0.1755 - accuracy: 0.9326
Epoch 7/20
39/39 [==============================] - 7s 175ms/step - loss: 0.1524 - accuracy: 0.9491
Epoch 8/20
39/39 [==============================] - 7s 184ms/step - loss: 0.1252 - accuracy: 0.9606
Epoch 9/20
39/39 [==============================] - 7s 182ms/step - loss: 0.1083 - accuracy: 0.9655
Epoch 10/20
39/39 [==============================] - 8s 199ms/step - loss: 0.0939 - accuracy: 0.969

In [49]:
model.evaluate(np.array(x_test),np.array(y_test_n))

17/17 [==============================] - 14s 90ms/step - loss: 1.0364 - accuracy: 0.7274


[1.0364288091659546, 0.7274472117424011]